# Finding the Kauzmann temperature

This notebook is a short example on how to use the thermodynamics module to find the Kauzmann temperature. The Kauzmann temperature is the temperature were the entropy of the stable crystalline phase is equal to the entropy of the liquid phase. In other words, it is the temperature were the excess entropy is zero.

This example assumes that GlassPy and all its dependencies are installed in the machine or virtual environment being used.

In [12]:
import numpy as np
import pandas as pd

from glasspy.thermodynamics.heatcapacity import heatCapacityFunFromTuple
from glasspy.thermodynamics.entropy import excessEntropyFun, KauzmannTemperature

## Case study: boron oxide (B<sub>2</sub>O<sub>3</sub>)

To compute the Kauzmann temperature we will need:
* The heat capacity at constant pressure (C<sub>p</sub>) of the stable crystalline phase
* The temperature and enthalpy of any phase transformation that occurs in the crystalline state
* The heat capacity at constant pressure (C<sub>p</sub>) of the liquid phase
* The melting temperature and melting enthalpy

The function "KauzmannTemperature" expects that the user provides the python functions for the C<sub>p</sub> of the crystal and the liquid. These are functions that have one argument for temperature (in Kelvin), and returns the value for C<sub>p</sub>. The user is free to define these functions the way that best suits them.

In this example we will make use of the "heatCapacityFunFromTuple" function provided by the thermodynamics.heatcapacity module in GlassPy. Please refer to the docstring of this function for more information.

Let's see it in action!

In [13]:
Cp_tuple_B2O3_liquid = (
    (0, None),
    (723, {
        'T0': 128.3,
        'T1': -3.01E-4,
    }),
    (3200, None),
)

Cp_liquid_fun = heatCapacityFunFromTuple(Cp_tuple_B2O3_liquid)

temperature_range = np.linspace(1, 5000, 20)
Cp_calc = Cp_liquid_fun(temperature_range)

print('Temperature\tCp_liquid')
for T, Cp in zip(temperature_range, Cp_calc):
    print(f'{T:.0f}\t\t{Cp:.2f}')

Temperature	Cp_liquid
1		nan
264		nan
527		nan
790		128.06
1053		127.98
1317		127.90
1580		127.82
1843		127.75
2106		127.67
2369		127.59
2632		127.51
2895		127.43
3158		127.35
3421		nan
3684		nan
3948		nan
4211		nan
4474		nan
4737		nan
5000		nan


As you can see, the function "heatCapacityFunFromTuple" received a tuple of tuples as an argument and returned a function called "Cp_liquid_fun". This function was then tested and only returned finite numbers in a restricted temperature range.

This happens because the default behavior of the "heatCapacityFunFromTuple" is to generate a function that do not extrapolate into regions were we have no Cp information. This can be easily seen from the tuple of tuples "Cp_tuple_B2O3_liquid" that we used to feed the function. See, this tuple

    Cp_tuple_B2O3_liquid = (
        (0, None),
        (723, {
            'T0': 128.3,
            'T1': -3.01E-4,
        }),
        (3200, None),
    )

is telling the function that from 0 to 723 Kelvin there is no Cp function, as well as from 3200 upwards. This can be checked by the first item in the "subtuples" inside. You can see from the previous example that this is strictly followed when using the generated function.

Now observe this subtuple:

    (723, {
        'T0': 128.3,
        'T1': -3.01E-4,
    }),
    
The second item in this subtuple is a dictionary that indicates the coefficients of the Cp function that is valid from the temperature 723 onwards (in this case, up to the temperature of the next subtuple, that is 3200). In this case we have only two coefficients: 'T0' and 'T1'. The resulting function that will be build with this information is:


    def function(T):
        return -3.01e-4*T + 128.3

You see, 'T0' is the temperature-independent term (equivalent to multiplying the temperature to the power of zero), and 'T1' is the coefficient that multiplies the temperature to the power of 1. The valid coefficient names are:

    'T0': coefficient independent of temperature
    'T1': coefficient that multiplies T
    'T2': coefficient that multiplies T**2
    'T3': coefficient that multiplies T**3
    'T4': coefficient that multiplies T**4
    '_T1': coefficient that multiplies T**(-1)
    '_T2': coefficient that multiplies T**(-2)
    '_T3': coefficient that multiplies T**(-3)
    '_T1_2': coefficient that multiplies T**(-1/2)

We will see some of these coefficients moving forward in this example.

If we want to extrapolate the Cp values outside the regions were we have data we can pass the argument 'extrapolate_borders=True'. Let's see what happens:

In [14]:
Cp_tuple_B2O3_liquid = (
    (0, None),
    (723, {
        'T0': 128.3,
        'T1': -3.01E-4,
    }),
    (3200, None),
)

Cp_liquid_fun = heatCapacityFunFromTuple(Cp_tuple_B2O3_liquid, extrapolate_borders=True)

temperature_range = np.linspace(1, 5000, 20)
Cp_calc = Cp_liquid_fun(temperature_range)

print('Temperature\tCp_liquid')
for T, Cp in zip(temperature_range, Cp_calc):
    print(f'{T:.0f}\t\t{Cp:.2f}')

Temperature	Cp_liquid
1		128.30
264		128.22
527		128.14
790		128.06
1053		127.98
1317		127.90
1580		127.82
1843		127.75
2106		127.67
2369		127.59
2632		127.51
2895		127.43
3158		127.35
3421		127.27
3684		127.19
3948		127.11
4211		127.03
4474		126.95
4737		126.87
5000		126.80


Now the Cp can be computed in all temperature ranges. This may be dangerous! In this case, for example, the Cp of the liquid does not approach zero as the temperature approaches the absolute zero. 

Next, we do tha same for the crystal heat capacity:

In [15]:
Cp_tuple_B2O3_crystal = (
    (0, {
        'T2': -4.73319e-05,
        'T3': 5.62398e-05,
    }),
    (27.64, {
        'T0': -10.10848,
        'T1': 0.26871,
        'T2': 0.00106,
        'T3': -6.65459e-06,
        '_T2': 2416.58355,
    }),
    (75.78, {
        'T0': -5.54677,
        'T1': 0.31337,
        'T2': -0.000411339,
        'T3': 4.37884e-07,
        '_T2': -12369.67768,
    }),
    (300, {
        'T0': 64.157456,
        'T1': 0.064596776,
        '_T2': -1836357.6,
        'T2': 0.000000033472,
    }),
    (723, None),
)

Cp_tuple_B2O3_liquid = (
    (0, None),
    (723, {
        'T0': 128.3,
        'T1': -3.01E-4,
    }),
    (3200, None),
)

Cp_crystal_fun = heatCapacityFunFromTuple(Cp_tuple_B2O3_crystal)

temperature_range = np.linspace(1, 1000, 20)
Cp_calc = Cp_crystal_fun(temperature_range)

print('Temperature\tCp_crystal')
for T, Cp in zip(temperature_range, Cp_calc):
    print(f'{T:.0f}\t\t{Cp:.2f}')

Temperature	Cp_crystal
1		0.00
54		7.15
106		22.51
159		35.09
211		46.16
264		56.37
316		66.27
369		74.52
422		81.07
474		86.63
527		91.58
579		96.12
632		100.39
685		104.47
737		nan
790		nan
842		nan
895		nan
947		nan
1000		nan


We only need to define the melting point (Tm) and the enthalpy of melting (delta_Hm). Fortunately, boron oxide does not have a crystalline phase transition, so the variable "H_transformations" is set no None

In [16]:
Cp_crystal_fun = heatCapacityFunFromTuple(Cp_tuple_B2O3_crystal)

Tm = 723
delta_Hm = 24070.0
H_transformations = None

Now we can compute the Kauzmann temperature.

In [17]:
TK = KauzmannTemperature(Cp_crystal_fun, 
                         Cp_liquid_fun, 
                         Tm, 
                         delta_Hm,
                         H_transformations)

print(f'The Kauzmann temperature of B2O3 is {TK:.2f} Kelvin.')

The Kauzmann temperature of B2O3 is 320.20 Kelvin.


## Case study: silica (SiO<sub>2</sub>) --- Phase tranformations

The procedure for silica is the same as shown for boron oxide. However silica differs from boron oxide due to the presence of crystalline phase transformations. Let's define everything else first.

In [18]:
Cp_tuple_SiO2_crystal = (
    (0, {
        'T2': 0.00308,
        'T3': -1.5072E-5
    }, 'Low quartz'),
    (100, {
        'T0': -7.55755,
        'T1': 0.26927,
        'T2': -3.9187E-4,
        'T3': 2.49689E-7
    }, 'Low quartz'),
    (298.15, {
        'T0': 58.082288,
        'T1': -0.000033472,
        '_T2': -1425907.2,
        'T2': 0.00002822108
    }, 'Low quartz'),
    (847, {
        'T0': 58.873064,
        'T1': 0.010070888,
        '_T2': 11715.2
    }, 'High quartz'),
    (1079, {
        'T0': 72.734656,
        'T1': 0.001330512,
        '_T2': -4128771.2,
        'T2': -0.000000012552
    }, 'High cristobalite'),
    (1996, None),
)

Cp_tuple_SiO2_liquid = (
    (0, None),
    (1480, {
        'T0': 81.373,
    }),
    (2000, None),
)

Cp_crystal_fun = heatCapacityFunFromTuple(Cp_tuple_SiO2_crystal)
Cp_liquid_fun = heatCapacityFunFromTuple(Cp_tuple_SiO2_liquid, extrapolate_borders=True)
Tm = 1999.0
delta_Hm = 8920.0

Silica has the transformation from low quarts to high quartz at 847 Kelvin, with an enthalpy of 728 J/mol during heating; and the transformation from high quartz to high cristobalite at 1079 Kelvin, with an enthalpy of 1998.8 J/mol during heating.

This information must be presented to the "KauzmannTemperature" function as a dictonary where the keys are the temperatures of transformation and the values of each key is the enthalpy of transformation. For silica this is the expected heat of transformation dictionary:

In [19]:
H_of_transformations_SiO2 = {
    847: 728.0,  # from low quartz to high quartz (J/mol)
    1079: 1995.8,  # from high quartz to high cristobalite (J/mol)
}

Finally, we can compute the Kauzmann temperature!

In [20]:
TK = KauzmannTemperature(Cp_crystal_fun, 
                         Cp_liquid_fun, 
                         Tm, 
                         delta_Hm,
                         H_of_transformations_SiO2)

print(f'The Kauzmann temperature of SiO2 is {TK:.2f} Kelvin.')

The Kauzmann temperature of SiO2 is nan Kelvin.


... 

What happened? It seems that silica does not have a Kauzmann temperature. It is possible that some materials will not have a Kauzmann temperature, but another explanation is that we are having problems with the domain of the heat capacity functions. If you look at the tuple "Cp_tuple_SiO2_crystal" you will see that the heat capacity is only defined up to 1996 Kelvin. However, the melting point of silica was defined as 1999 K, and we need to have a well defined heat capacity between the melting point and the eventual Kauzmann tempetature, otherwise no Kauzmann temperature will be found.

This can be fixed by using the "extrapolate_borders=True" argument.

In [21]:
Cp_crystal_fun = heatCapacityFunFromTuple(Cp_tuple_SiO2_crystal, extrapolate_borders=True)

And now everything should work fine.

In [22]:
TK = KauzmannTemperature(Cp_crystal_fun, 
                         Cp_liquid_fun, 
                         Tm, 
                         delta_Hm,
                         H_of_transformations_SiO2)

print(f'The Kauzmann temperature of SiO2 is {TK:.2f} Kelvin.')

The Kauzmann temperature of SiO2 is 1178.12 Kelvin.
